In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [25]:
df = spark.read.format('csv').option("header", "True").load("data/cars.csv")

In [26]:
df.show(10)

+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------------+
|manufacturer_name|model_name|transmission| color|odometer_value|year_produced|engine_fuel|engine_has_gas|engine_type|engine_capacity|body_type|has_warranty|state|drivetrain|price_usd|is_exchangeable|location_region|number_of_photos|up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|duration_listed|
+-----------------+----------+------------+------+--------------+-------------+-----------+--------------+-----------+---------------+---------+------------+-----+----------+---------+---------------+---------------+----------------+----------+---------+---------+---------+----

In [27]:
df.show(1, vertical=True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

## Методы DataFrame API

### .select()

Работает в SQL

In [28]:
df.select("manufacturer_name", "model_name").show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



In [29]:
import pyspark.sql.functions as F

df.select(F.col("manufacturer_name"), F.col("model_name")).show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
|           Subaru|   Impreza|
|           Subaru|    Legacy|
+-----------------+----------+
only showing top 5 rows



### .filter()

In [30]:
# выберем только марки Audi

df.select("manufacturer_name", "model_name").filter("manufacturer_name = 'Audi'").show(5)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|             Audi|        Q7|
|             Audi|        TT|
|             Audi|       100|
|             Audi|        A6|
|             Audi|        Q3|
+-----------------+----------+
only showing top 5 rows



In [31]:
# все Audi с ручной коробкой
df.select("manufacturer_name", "model_name", "transmission")\
.filter("manufacturer_name = 'Audi' and transmission = 'mechanical'").show(5)

+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



In [32]:
# лучше использовать col для совершения условий фильтрации
df\
    .select("manufacturer_name", "model_name", "transmission")\
    .filter(F.col("manufacturer_name") == "Audi" )\
    .filter(F.col("transmission") == 'mechanical').show(5)


+-----------------+----------+------------+
|manufacturer_name|model_name|transmission|
+-----------------+----------+------------+
|             Audi|       100|  mechanical|
|             Audi|A6 Allroad|  mechanical|
|             Audi|       100|  mechanical|
|             Audi|        A4|  mechanical|
|             Audi|        80|  mechanical|
+-----------------+----------+------------+
only showing top 5 rows



### .count()

возвращает число строк

In [33]:
df.count()

38531

In [34]:
# посчитать уникальные 
df.distinct().count()

38491

In [35]:
df.select("manufacturer_name").distinct().count()

55

### .groupBy() and orderBy()

In [36]:
# сгруппировать по manufacturer_name и посчитать количество каждого
df.groupBy('manufacturer_name').count().show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|            Lexus|  213|
|           Jaguar|   53|
|            Rover|  235|
|           Lancia|   92|
+-----------------+-----+
only showing top 5 rows



Сортировка по колонке по убыванию. Тут надо явно задать колонку через col():

In [37]:
df.groupBy('manufacturer_name').count().orderBy(F.col("count").desc()).show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



### .witColumnRenamed() and .withColumn() 

Переменовать существующую колонку колонку:

In [38]:
df = df.withColumnRenamed("manufacturer_name", "manufacturer")

In [39]:
df.select("manufacturer").show(5)

+------------+
|manufacturer|
+------------+
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
|      Subaru|
+------------+
only showing top 5 rows



Создаем новую колонку

In [40]:
df.withColumn("next_year", F.col("year_produced") + 1).select('next_year').show(5)

+---------+
|next_year|
+---------+
|   2011.0|
|   2003.0|
|   2002.0|
|   2000.0|
|   2002.0|
+---------+
only showing top 5 rows



### .printSchema() and .describe()

Выводить схему и типы колонок: 

In [41]:
df.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)
 |-- 

In [42]:
df.describe().show()

+-------+------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+------------------+---------+------------+---------+----------+-----------------+---------------+----------------+-----------------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+
|summary|manufacturer|        model_name|transmission| color|    odometer_value|     year_produced|  engine_fuel|engine_has_gas|engine_type|   engine_capacity|body_type|has_warranty|    state|drivetrain|        price_usd|is_exchangeable| location_region| number_of_photos|        up_counter|feature_0|feature_1|feature_2|feature_3|feature_4|feature_5|feature_6|feature_7|feature_8|feature_9|   duration_listed|
+-------+------------+------------------+------------+------+------------------+------------------+-------------+--------------+-----------+------------------+---------+---------

In [43]:
# остановка системы
# spark.stop()